# Convoluntional Variational AutoEncoder
Reference: https://keras.io/examples/generative/vae/

My idea for generating new data samples labelled as Normal and Tumor is to train a VAE which can generate a Normal sample when given a Normal test sample and a Tumor sample when given a Tumor test sample.

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Create a sampling layer


In [6]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon



## Build the encoder


In [4]:
nt_test = np.load('data/nt_test.npy')

In [7]:
latent_dim = 2

encoder_inputs = keras.Input(shape=(nt_test.shape[1], 1))
x = layers.Conv1D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv1D(16, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()


Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 60483, 1)]   0                                            
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 30242, 32)    128         input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 15121, 16)    1552        conv1d_2[0][0]                   
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 241936)       0           conv1d_3[0][0]                   
____________________________________________________________________________________________

## Build the decoder


In [12]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(16 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((16, 64))(x)
x = layers.Conv1DTranspose(64, 3, activation="relu", strides=2, padding='same')(x)
x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding='same')(x)
x = layers.Conv1DTranspose(1, 3, activation='relu', padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(nt_test.shape[1])(x)
decoder_outputs = layers.Reshape((nt_test.shape[1], 1))(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              3072      
_________________________________________________________________
reshape_4 (Reshape)          (None, 16, 64)            0         
_________________________________________________________________
conv1d_transpose_12 (Conv1DT (None, 32, 64)            12352     
_________________________________________________________________
conv1d_transpose_13 (Conv1DT (None, 64, 32)            6176      
_________________________________________________________________
conv1d_transpose_14 (Conv1DT (None, 64, 1)             97        
_________________________________________________________________
flatten_5 (Flatten)          (None, 64)                0   

## Define the VAE as a `Model` with a custom `train_step`


In [67]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 28 * 28
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }



## Train the VAE


In [1]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(nt_test[:10], epochs=3, batch_size=1)

NameError: name 'VAE' is not defined